In [1]:
import pandas as pd
import math

## Data

Read in the data.
The amount per minute is amount created from a machine single machine in 1 minute.

In [2]:
recipe_df = pd.read_csv("Satisfactory Recipes - parts per minute.csv", index_col=0)

In [3]:
recipe_df.sample(2)

,Amount Per Minute,Iron Ore,Copper Ore,Caterium Ore,Iron Ingot,Copper Ingot,Caterium Ingot,Wire,Cable,Screw,Quickwire,Crude Oil,Rubber,Plastic,Circuit Board,High-Speed Connecter,A.I. Limiter,Computer
Item,,,,,,,,,,,,,,,,,,
Quickwire,90.0,NaN,NaN,NaN,NaN,15.0,7.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Circuit Board,7.5,NaN,NaN,NaN,NaN,NaN,NaN,60.0,NaN,NaN,NaN,NaN,40.0,NaN,NaN,NaN,NaN,NaN


### Data Prep

Seperate the amount per minute column from everything else and turn it into a dictionary.

The key is the item name, the value is a tuple of the (apm, recipe) where the recipe itself is a dict.

In [4]:
apm = recipe_df["Amount Per Minute"]
recipe_no_apm = recipe_df.drop("Amount Per Minute", axis = 1)

In [5]:
recipes = {item : (apm.loc[item], recipe_no_apm.loc[item].dropna().to_dict()) for item in recipe_no_apm.index}
recipes

{'Iron Ingot': (30.0, {'Iron Ore': 30.0}),
 'Copper Ingot': (30.0, {'Copper Ore': 30.0}),
 'Caterium Ingot': (15.0, {'Caterium Ore': 60.0}),
 'Wire': (45.0, {'Copper Ingot': 15.0}),
 'Cable': (15.0, {'Wire': 30.0}),
 'Screw': (90.0, {'Iron Ingot': 15.0}),
 'Quickwire': (90.0, {'Copper Ingot': 15.0, 'Caterium Ingot': 7.5}),
 'Rubber': (30.0, {'Crude Oil': 30.0}),
 'Plastic': (22.5, {'Crude Oil': 30.0}),
 'Circuit Board': (7.5, {'Wire': 60.0, 'Rubber': 40.0}),
 'A.I. Limiter': (5.0, {'Quickwire': 90.0, 'Circuit Board': 5.0}),
 'High-Speed Connecter': (2.5,
  {'Cable': 25.0, 'Quickwire': 100.0, 'Plastic': 15.0}),
 'Computer': (1.875,
  {'Cable': 22.5, 'Screw': 112.5, 'Plastic': 33.75, 'Circuit Board': 9.375}),
 'Super Computer': (1.875,
  {'Plastic': 39.375,
   'High-Speed Connecter': 5.625,
   'A.I. Limiter': 3.75,
   'Computer': 3.75})}

## Conversion

The first function is a util function for adding to the usage stats.

In [6]:
def add_used(used, usage_stats, recipe_apm):
    '''
    Util function for adding to the usage stats.
    
    This will not modify the original usage_stats value.
    '''
    usage_stats = {**usage_stats}
    usage_stats["used"] += used
    usage_stats["buildings"] = math.ceil(usage_stats["used"] / recipe_apm)
    usage_stats["gross"] = usage_stats["buildings"] * recipe_apm
    return usage_stats

The second function calculates how many secondary reagents are required to make the a certain amount of original item.

EG: 45 Wire requires 15 Copper Ingots which in turn requires 15 Copper Ore.
So this function will return 45 Wire -> 15 Copper Ore.

In [7]:
def calc_secondary_reagants(item, recipes, amount, use_ceil = False):
    '''
    Calculates how many secondary reagents are required to make the a certain amount of original item.
    
    EG: 45 Wire requires 15 Copper Ingots which in turn requires 15 Copper Ore.
        So this function will return 45 Wire -> 15 Copper Ore.
    '''
    if item not in recipes:
        return None
    res = {}
    apm1, input1 = recipes[item]
    conversion_ratio = math.ceil(amount/apm1) if use_ceil else amount/apm1
    for reagent, count in input1.items():
        if reagent not in recipes:
            continue
        apm2, _ = recipes[reagent]
        usage_stats = res.get(reagent, {"gross": 0, "used": 0, "buildings": 0})
        res[reagent] = add_used(input1[reagent] * conversion_ratio, usage_stats, apm2)
    return conversion_ratio, res

### Example

In this section I calculate what is required to build 3.75 (2 buildings) Super Computers per minute.

In [8]:
item_type = "Super Computer"
create = 1.875 * 2
buildings, cost = calc_secondary_reagants("Super Computer", recipes, create, use_ceil = False)
buildings = math.ceil(buildings)
print(f"{buildings} buildings to create {create} '{item_type}' per minute.")

2 buildings to create 3.75 'Super Computer' per minute.


Use a dequeue to calculate the number and variety of products required to create the Super Computers.

In [9]:
queue = [cost]
all_buildings = {item_type: {"gross": buildings * recipes[item_type][0], "used": create, "buildings": buildings}}
while queue:
    c = queue.pop(-1)
    for reagent, usage_stats in c.items():
        _, c2 = calc_secondary_reagants(reagent, recipes, usage_stats["used"])
        previous_usage_stats = all_buildings.get(reagent, {"gross": 0, "used": 0, "buildings": 0})
        all_buildings[reagent] = add_used(usage_stats["used"], previous_usage_stats, recipes[reagent][0])
        if c2:
            queue.append(c2)

In [10]:
all_buildings

{'Super Computer': {'gross': 3.75, 'used': 3.75, 'buildings': 2},
 'Plastic': {'gross': 292.5, 'used': 281.25, 'buildings': 13},
 'High-Speed Connecter': {'gross': 12.5, 'used': 11.25, 'buildings': 5},
 'A.I. Limiter': {'gross': 10.0, 'used': 7.5, 'buildings': 2},
 'Computer': {'gross': 7.5, 'used': 7.5, 'buildings': 4},
 'Cable': {'gross': 210.0, 'used': 202.5, 'buildings': 14},
 'Screw': {'gross': 450.0, 'used': 450.0, 'buildings': 5},
 'Circuit Board': {'gross': 45.0, 'used': 45.0, 'buildings': 6},
 'Wire': {'gross': 765.0, 'used': 765.0, 'buildings': 17},
 'Rubber': {'gross': 240.0, 'used': 240.0, 'buildings': 8},
 'Copper Ingot': {'gross': 360.0, 'used': 352.5, 'buildings': 12},
 'Iron Ingot': {'gross': 90.0, 'used': 75.0, 'buildings': 3},
 'Quickwire': {'gross': 630.0, 'used': 585.0, 'buildings': 7},
 'Caterium Ingot': {'gross': 60.0, 'used': 48.75, 'buildings': 4}}